In [153]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import warnings
#warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
#import gensim
#from gensim.models.doc2vec import Doc2Vec, LabeledSentence 
import numpy as np
import copy
import time
%matplotlib inline

# 读数据 concat

In [2]:
def load_data():
    train_ori = pd.read_table("round1_ijcai_18_train_20180301.txt",delim_whitespace=True)
    test_ori = pd.read_table('round1_ijcai_18_test_a_20180301.txt',delim_whitespace = True)
    return train_ori, test_ori

In [3]:
train_ori, test_ori = load_data()
train_ori = train_ori.drop_duplicates()#训练集去重
train_1 = train_ori.copy(deep = True)
test_1 = test_ori.copy(deep = True)

In [4]:
print(test_ori.shape)
print(test_1.shape)
print(train_ori.shape)
print(train_1.shape)

(18371, 26)
(18371, 26)
(478111, 27)
(478111, 27)


In [5]:
train_y = train_1["is_trade"]
train_1 = train_1.drop("is_trade",axis = 1)

In [6]:
data = pd.concat([train_1,test_1])
data = data.reset_index(drop = True)
print(data.shape)

(496482, 26)


# 处理item_category_list这个feature

In [7]:
data.head(3)

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,context_timestamp,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537236544,4006,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537243232,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537211052,4001,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0


In [8]:
cate_list = list(data['item_category_list'])

In [9]:
# #分号前的category1
# cate_1_list = []
# for i in range(len(cate_list)):
#     cate_1_list.append(cate_list[i].split(";")[0])
# cate_1_list[:3]

In [10]:
# set(cate_1_list)#发现所有样本都一样 因此去掉该子字段的信息

In [11]:
#分号后的category2
cate_2_list = []
for i in range(len(cate_list)):
    cate_2_list.append(cate_list[i].split(";")[1])

In [12]:
# set(cate_2_list)#有区别 因此用该子字段的信息代替原来的feature

In [13]:
data["item_category_list"] = cate_2_list

In [14]:
data.head(3)

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,context_timestamp,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description
0,108641074714126964,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537236544,4006,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0
1,5754713551599725161,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537243232,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0
2,842679481291040981,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537211052,4001,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0


In [15]:
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return dt

In [16]:
def convert_data(data):
    data['time'] = data.context_timestamp.apply(timestamp_datetime)
    data['day'] = data.time.apply(lambda x: int(x[8:10]))
    data['hour'] = data.time.apply(lambda x: int(x[11:13]))
    user_query_day = data.groupby(['user_id', 'day']).size(
    ).reset_index().rename(columns={0: 'user_query_day'})
    data = pd.merge(data, user_query_day, 'left', on=['user_id', 'day'])
    user_query_day_hour = data.groupby(['user_id', 'day', 'hour']).size().reset_index().rename(
        columns={0: 'user_query_day_hour'})
    data = pd.merge(data, user_query_day_hour, 'left',
                    on=['user_id', 'day', 'hour'])

    return data

In [18]:
data = convert_data(data)

In [19]:
data = data.drop("time",axis = 1)
data

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,day,hour,user_query_day,user_query_day_hour
0,108641074714126964,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4,1.000000,5002,1.000000,1.000000,1.000000,18,10,3,3
1,5754713551599725161,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4,1.000000,5002,1.000000,1.000000,1.000000,18,12,8,5
2,842679481291040981,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4,1.000000,5002,1.000000,1.000000,1.000000,18,3,2,2
3,937088850059189027,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4,1.000000,5002,1.000000,1.000000,1.000000,18,6,1,1
4,7975697065017708072,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4,1.000000,5002,1.000000,1.000000,1.000000,18,19,2,2
5,7764762765372067286,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4,1.000000,5002,1.000000,1.000000,1.000000,18,23,4,3
6,6956333474094867789,285660928590172217,8277336076276184272,2072967855524022579;5131280576272319091;263639...,9057103201734987852,548352491538518780,8,9,8,13,...,15,0.985427,5012,0.974878,0.976863,0.969278,18,22,23,4
7,8387099821892927911,285660928590172217,8277336076276184272,2072967855524022579;5131280576272319091;263639...,9057103201734987852,548352491538518780,8,9,8,13,...,15,0.985427,5012,0.974878,0.976863,0.969278,18,16,2,1
8,4021878205550012615,5202355029344881809,5755694407684602296,2072967855524022579;5131280576272319091;263639...,5520678735822176314,548352491538518780,8,9,10,16,...,15,0.985427,5012,0.974878,0.976863,0.969278,18,2,1,1
9,6499571365974135517,285660928590172217,8277336076276184272,2072967855524022579;5131280576272319091;263639...,9057103201734987852,548352491538518780,8,9,8,13,...,15,0.985427,5012,0.974878,0.976863,0.969278,18,23,4,4


# item_property_list这个feature与pred_property这个集合取交集 再除以真实property_list集合的势

In [20]:
data.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'day', 'hour', 'user_query_day',
       'user_query_day_hour'],
      dtype='object')

In [21]:
pred_cate_prop_list = list(data['predict_category_property'])
#用；分解每个cate：property
pcp_list = []
for i in pred_cate_prop_list:
    pcp_list.append(i.split(';'))
#再用：将cate，property分开
pcp_list_2 = []
for i in pcp_list:
    tmp = []
    for k in i:
        tmp.append(k.split(":"))
    pcp_list_2.append(tmp)
#每一行的category
each_category_list = []
for i in pcp_list_2:
    tmp = []
    for k in i:
        tmp.append(k[0])
    each_category_list.append(tmp)
#每一行的property
each_property_list = []
for i in pcp_list_2:
    tmp = []
    for k in i:
        tmp.extend(k[1:])
    each_property_list.append(tmp)
def property_clean(doc):
    doc_trans = []
    for i in range(len(doc)):
        temp = []
        for j in range(len(doc[i])):
            for k in range(len(doc[i][j].split(","))):
                temp.append(doc[i][j].split(",")[k])
        doc_trans.append(temp)
    return doc_trans
property_clean_1 = property_clean(each_property_list)
each_property_list_prep = [list(set(x)) for x in property_clean_1]
print(each_category_list[:3])#预测的类别
print("======================")
print(each_property_list_prep[:3])#预测的属性

[['5799347067982556520', '509660095530134768', '5755694407684602296', '8277336076276184272', '7908382889764677758'], ['5799347067982556520', '7908382889764677758'], ['5799347067982556520', '7258015885215914736', '7908382889764677758']]
[['-1', '9148482949976129397'], ['9172976955054793469', '5195139481388729954', '9148482949976129397', '4621934203383159480', '1787573075717641245'], ['5131280576272319091']]


In [27]:
for i in each_property_list_prep:
    if '-1' in i:
        i.remove('-1')
print(each_property_list_prep[:3])#预测的属性

[['9148482949976129397'], ['9172976955054793469', '5195139481388729954', '9148482949976129397', '4621934203383159480', '1787573075717641245'], ['5131280576272319091']]


In [28]:
lenth_cate_pred = []
for i in range(len(each_category_list)):
    lenth_cate_pred.append(len(each_category_list[i]))
lenth_prop_pred = []
for i in range(len(each_property_list_prep)):
    lenth_prop_pred.append(len(each_property_list_prep[i]))

In [29]:
print(lenth_cate_pred[:2])
print(lenth_prop_pred[:2])

[5, 2]
[1, 5]


In [30]:
data['lenth_cate_pred'] = lenth_cate_pred#int
data['lenth_prop_pred'] = lenth_prop_pred#int

In [33]:
complexity_list = []
for i in range(len(data)):
    complexity_list.append(data.loc[i,'lenth_cate_pred'] * data.loc[i,'lenth_prop_pred'])
data['query_complexity'] = complexity_list

In [34]:
complexity_list[:2]

[5, 10]

In [35]:
union_property = []
for i in range(len(data)):
    union_property.append(len((set(data.loc[i,"item_property_list"].split(";"))) & \
     set(each_property_list_prep[i]))/len(set(data.loc[i,"item_property_list"].split(";"))))

In [36]:
lenth_prop = []
for i in range(len(data)):
    lenth_prop.append(len(set(data.loc[i,"item_property_list"].split(";"))))

In [37]:
data["union_property"] = union_property
data["lenth_prop"] = lenth_prop

In [38]:
union_category = []
for i in range(len(data)):
    union_category.append(len((set([data.loc[i,"item_category_list"]])) & \
     set(each_category_list[i])))

In [39]:
data["union_category"] = union_category
#data["lenth_cate"] = lenth_cate

In [40]:
#添加word2vec的特征
data_w2v = pd.read_csv("data_nlp.csv")

In [42]:
w2v_features = ['prop1', 'prop2', 'prop3', 'prop4', 'prop5',
       'prop6', 'prop7', 'prop8', 'prop9', 'prop10', 'pred_cate1',
       'pred_cate2', 'pred_cate3', 'pred_cate4', 'pred_cate5', 'pred_prop1',
       'pred_prop2', 'pred_prop3', 'pred_prop4', 'pred_prop5', 'pred_prop6',
       'pred_prop7', 'pred_prop8', 'pred_prop9', 'pred_prop10']

In [43]:
data_ = pd.concat([data,data_w2v[w2v_features]],axis = 1)

In [45]:
data_ = data_.drop(["item_property_list","predict_category_property"],axis = 1)

In [46]:
data_.head(3)

,instance_id,item_id,item_category_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,...,pred_prop1,pred_prop2,pred_prop3,pred_prop4,pred_prop5,pred_prop6,pred_prop7,pred_prop8,pred_prop9,pred_prop10
0,108641074714126964,3412720377098676069,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,4505772604969228686,...,-0.087757,0.152223,-0.224348,0.068927,-0.110023,-0.021113,-0.184340,0.532930,-0.222828,0.020902
1,5754713551599725161,3412720377098676069,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,2692638157208937547,...,-0.213730,-0.006777,-0.040061,-0.092147,-0.126459,0.002954,-0.030236,0.002456,-0.190004,0.070656
2,842679481291040981,3412720377098676069,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,5247924392014515924,...,-0.062296,-0.013683,-0.030650,-0.054671,-0.029541,-0.003900,0.029093,0.019030,0.055204,-0.015705


In [47]:
# #分号分割后的所有属性放到一起
# property_list = list(data['item_property_list'])
# property_list_all = []
# for i in range(len(property_list)):
#     for j in range(len(property_list[i].split(";"))):
#         property_list_all.append(property_list[i].split(";")[j])
# #看看有多少种不同的属性
# property_distinct = list(set(property_list_all))
# print(len(property_distinct))

In [48]:
# #将原始的数据转换成doc2vec可用的lists of list的格式
# property_list_pre = []
# for i in range(len(property_list)):
#     temp = []
#     temp.append(property_list[i])
#     property_list_pre.append(temp)
# pre_process = [[word for word in document[0].split(';')] for document in property_list_pre]

In [49]:
# #看看拆分结果
# print(property_list_pre[:2])
# print("-"*50)
# print(pre_process[:2])

In [50]:
# def doc2vec(doc, size, window = 1, min_count = 1):
#     """
#     doc:lists of list格式
#     size:目标vector大小 
#     """
#     TaggededDocument = gensim.models.doc2vec.TaggedDocument  
#     labeled_doc = []  
#     for i, text in enumerate(doc):  
#         document = TaggededDocument(text, tags=[i])  
#         labeled_doc.append(document)
#     model_dm = Doc2Vec(labeled_doc, min_count = min_count, window = window, size = size, 
#                        negative=5, workers=4)
#     array_property = model_dm.infer_vector(pre_process[0])
#     for i in doc[1:]:
#         temp = model_dm.infer_vector(i)
#         array_property = np.vstack((array_property,temp))
#     dataframe_property = pd.DataFrame(array_property)
#     return dataframe_property

In [51]:
# dataframe_property = doc2vec(pre_process, 10)
# dataframe_property.head(3)

In [52]:
# dataframe_property.shape

In [53]:
# data.shape

In [54]:
# dataframe_property.columns = ["prop1","prop2","prop3","prop4","prop5",
#                              "prop6","prop7","prop8","prop9","prop10"]

In [55]:
# data_ = pd.concat([data, dataframe_property],axis=1)

In [56]:
# len(set(list(data["item_brand_id"])))#商品品牌暂时不处理

In [57]:
data_1 = data_.drop(['instance_id','item_id','user_id'],axis = 1)
data_1.head(3)

,item_category_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_gender_id,user_age_level,user_occupation_id,...,pred_prop1,pred_prop2,pred_prop3,pred_prop4,pred_prop5,pred_prop6,pred_prop7,pred_prop8,pred_prop9,pred_prop10
0,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,1,1003,2005,...,-0.087757,0.152223,-0.224348,0.068927,-0.110023,-0.021113,-0.184340,0.532930,-0.222828,0.020902
1,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,0,1002,2005,...,-0.213730,-0.006777,-0.040061,-0.092147,-0.126459,0.002954,-0.030236,0.002456,-0.190004,0.070656
2,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,0,1003,2005,...,-0.062296,-0.013683,-0.030650,-0.054671,-0.029541,-0.003900,0.029093,0.019030,0.055204,-0.015705


In [58]:
data_1.shape

(496482, 56)

# 处理predict_category_property这个feature

In [59]:
#pred_cate_prop_list = list(data_1['predict_category_property'])

In [60]:
#pred_cate_prop_list[:3]

In [61]:
# #用；分解每个cate：property
# pcp_list = []
# for i in pred_cate_prop_list:
#     pcp_list.append(i.split(';'))
# #再用：将cate，property分开
# pcp_list_2 = []
# for i in pcp_list:
#     tmp = []
#     for k in i:
#         tmp.append(k.split(":"))
#     pcp_list_2.append(tmp)

In [62]:
# pcp_list_2[:2]

In [63]:
# #每一行的category
# each_category_list = []
# for i in pcp_list_2:
#     tmp = []
#     for k in i:
#         tmp.append(k[0])
#     each_category_list.append(tmp)

In [64]:
# each_category_list[:3]

In [65]:
# dataframe_pred_cate = doc2vec(each_category_list, 5)
# dataframe_pred_cate.head(3)

In [66]:
# dataframe_pred_cate.columns = ["pred_cate1","pred_cate2","pred_cate3",
#                                "pred_cate4","pred_cate5"]

In [67]:
# data_1_ = pd.concat([data_1, dataframe_pred_cate],axis=1)
# data_1_.head(3)

In [68]:
# data_1_.shape

In [69]:
# #每一行的property
# each_property_list = []
# for i in pcp_list_2:
#     tmp = []
#     for k in i:
#         tmp.extend(k[1:])
#     each_property_list.append(tmp)

In [70]:
# each_property_list[:2]

In [71]:
# def property_clean(doc):
#     doc_trans = []
#     for i in range(len(doc)):
#         temp = []
#         for j in range(len(doc[i])):
#             for k in range(len(doc[i][j].split(","))):
#                 temp.append(doc[i][j].split(",")[k])
#         doc_trans.append(temp)
#     return doc_trans

In [72]:
# def property_clean(doc):
#     doc_trans = []
#     for i in range(len(doc)):
#         temp = []
#         for j in range(len(doc[i])):
#             for k in range(len(doc[i][j].split(","))):
#                 temp.append(doc[i][j].split(",")[k])
#         doc_trans.append(temp)
#     return doc_trans
# property_clean_1 = property_clean(each_property_list)
# property_clean_1[:2]

In [73]:
# def property_clean(doc):
#     doc_trans = []
#     for i in range(len(doc)):
#         temp = []
#         for j in range(len(doc[i])):
#             for k in range(len(doc[i][j].split(","))):
#                 temp.append(doc[i][j].split(",")[k])
#         doc_trans.append(temp)
#     return doc_trans
# property_clean_1 = property_clean(each_property_list)
# property_clean_1[:2]
# each_property_list_prep = [list(set(x)) for x in property_clean_1]
# each_property_list_prep[:2]

In [74]:
# dataframe_pred_prop = doc2vec(each_property_list_prep, 10)
# dataframe_pred_prop.head(3)

In [75]:
# dataframe_pred_prop.columns = ["pred_prop1","pred_prop2","pred_prop3","pred_prop4","pred_prop5",
#                               "pred_prop6","pred_prop7","pred_prop8","pred_prop9","pred_prop10",]
# data_3 = pd.concat([data_2, dataframe_pred_prop],axis=1)
# data_3.head(3)

In [76]:
# data_3.shape

In [77]:
# data_3.to_csv("data_nlp.csv")

-----

# context_timestamp特征处理

In [78]:
# data_3 = pd.read_csv("data_nlp.csv")

In [79]:
def time_value(time_stamp):
    hour = int(time_stamp.strftime("%H:%M:%S")[:2]) * 3600
    minus = int(time_stamp.strftime("%H:%M:%S")[3:5]) * 60
    sec = int(time_stamp.strftime("%H:%M:%S")[6:8])
    return hour + minus + sec

In [80]:
import datetime 
time_value_list = []
day_list = []
for i in range(len(data_1['context_timestamp'])):
    time_stamp = datetime.datetime.utcfromtimestamp(data_1['context_timestamp'][i])
    day = time_stamp.strftime("%Y-%m-%d")[-2:]
    time_value_list.append(time_value(time_stamp))
    day_list.append(day)

In [81]:
data_1['context_timestamp'] = time_value_list
#data_1["day"] = day_list
data_1.head(3)

,item_category_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_gender_id,user_age_level,user_occupation_id,...,pred_prop1,pred_prop2,pred_prop3,pred_prop4,pred_prop5,pred_prop6,pred_prop7,pred_prop8,pred_prop9,pred_prop10
0,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,1,1003,2005,...,-0.087757,0.152223,-0.224348,0.068927,-0.110023,-0.021113,-0.184340,0.532930,-0.222828,0.020902
1,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,0,1002,2005,...,-0.213730,-0.006777,-0.040061,-0.092147,-0.126459,0.002954,-0.030236,0.002456,-0.190004,0.070656
2,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,0,1003,2005,...,-0.062296,-0.013683,-0.030650,-0.054671,-0.029541,-0.003900,0.029093,0.019030,0.055204,-0.015705


In [82]:
data_2 = data_1.copy(deep = True)

In [83]:
#增加一列 timestamp bin
#data_2['context_timestamp_bin'] = pd.cut(list(data_2['context_timestamp']),24)

In [84]:
data_2.columns

Index(['item_category_list', 'item_brand_id', 'item_city_id',
       'item_price_level', 'item_sales_level', 'item_collected_level',
       'item_pv_level', 'user_gender_id', 'user_age_level',
       'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'shop_id',
       'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level',
       'shop_score_service', 'shop_score_delivery', 'shop_score_description',
       'day', 'hour', 'user_query_day', 'user_query_day_hour',
       'lenth_cate_pred', 'lenth_prop_pred', 'query_complexity',
       'union_property', 'lenth_prop', 'union_category', 'prop1', 'prop2',
       'prop3', 'prop4', 'prop5', 'prop6', 'prop7', 'prop8', 'prop9', 'prop10',
       'pred_cate1', 'pred_cate2', 'pred_cate3', 'pred_cate4', 'pred_cate5',
       'pred_prop1', 'pred_prop2', 'pred_prop3', 'pred_prop4', 'pred_prop5',
       'pred_prop6', 'pred_prop7', 'pred_prop8', 'pred_prop9', 'pred_prop10'],
      dtype='

In [85]:
data_2.shape

(496482, 56)

In [86]:
#进一步去掉无用的feature
data_3 = data_2.drop(['context_id','shop_id','day'],axis = 1)

In [108]:
level_features = ["item_price_level","item_sales_level","item_collected_level",
                 "shop_review_num_level","item_pv_level","user_star_level",
                  "shop_star_level",'context_page_id']

In [109]:
data_3[level_features].head(3)

,item_price_level,item_sales_level,item_collected_level,shop_review_num_level,item_pv_level,user_star_level,shop_star_level,context_page_id
0,3,3,4,4,14,3003,5002,4006
1,3,3,4,4,14,3006,5002,4001
2,3,3,4,4,14,3004,5002,4001


In [94]:
sales_price_ratio = []
for i in range(len(data_3)):
    sales_price_ratio.append(data_3.loc[i,"item_sales_level"] / data_3.loc[i,"item_price_level"])
sales_price_ratio[:2]    

i:\anaconda\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in longlong_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


[1.0, 1.0]

In [96]:
data_3["sales_price_ratio"] = sales_price_ratio

In [97]:
continu_features = ['shop_score_description','shop_score_delivery',
               'shop_score_service','shop_review_positive_rate']

In [98]:
data_3[continu_features].head(3)

,shop_score_description,shop_score_delivery,shop_score_service,shop_review_positive_rate
0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0


In [99]:
data_3.columns

Index(['item_category_list', 'item_brand_id', 'item_city_id',
       'item_price_level', 'item_sales_level', 'item_collected_level',
       'item_pv_level', 'user_gender_id', 'user_age_level',
       'user_occupation_id', 'user_star_level', 'context_timestamp',
       'context_page_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'hour', 'user_query_day',
       'user_query_day_hour', 'lenth_cate_pred', 'lenth_prop_pred',
       'query_complexity', 'union_property', 'lenth_prop', 'union_category',
       'prop1', 'prop2', 'prop3', 'prop4', 'prop5', 'prop6', 'prop7', 'prop8',
       'prop9', 'prop10', 'pred_cate1', 'pred_cate2', 'pred_cate3',
       'pred_cate4', 'pred_cate5', 'pred_prop1', 'pred_prop2', 'pred_prop3',
       'pred_prop4', 'pred_prop5', 'pred_prop6', 'pred_prop7', 'pred_prop8',
       'pred_prop9', 'pred_prop10', 'sales_price_ratio'],
      dtype='object')

In [110]:
# feature_dummy = ["context_timestamp_bin","user_occupation_id","user_age_level",
#                 "user_gender_id",'item_city_id','item_category_list',"day",
#                  "item_pv_level","user_star_level","context_page_id","shop_review_num_level",
#                 "shop_star_level","item_sales_level","item_collected_level","item_price_level"]

feature_dummy = ["hour","user_occupation_id","user_age_level",
                "user_gender_id",'item_city_id','item_category_list',"user_star_level",
                 "shop_star_level","context_page_id"
                ]

In [111]:
# data_3[feature_dummy] = data_3[feature_dummy].astype(str)

In [112]:
data_3["user_occupation_id"] = data_3["user_occupation_id"].astype(str)
data_3["user_age_level"] = data_3["user_age_level"].astype(str)
data_3["user_gender_id"] = data_3["user_gender_id"].astype(str)
data_3['item_city_id'] = data_3['item_city_id'].astype(str)
data_3['item_category_list'] = data_3['item_category_list'].astype(str)
data_3['hour'] = data_3['hour'].astype(str)
data_3["user_star_level"] = data_3["user_star_level"].astype(str)
data_3["shop_star_level"] = data_3["shop_star_level"].astype(str)
data_3["context_page_id"] = data_3["context_page_id"].astype(str)

# data_3["item_pv_level"] = data_3["item_pv_level"].astype(str)
# data_3["user_star_level"] = data_3["user_star_level"].astype(str)
# data_3["context_page_id"] = data_3["context_page_id"].astype(str)
# data_3["shop_review_num_level"] = data_3["shop_review_num_level"].astype(str)
# data_3["shop_star_level"] = data_3["shop_star_level"].astype(str)
# data_3["item_sales_level"] = data_3["item_sales_level"].astype(str)
# data_3["item_collected_level"] = data_3["item_collected_level"].astype(str)
# data_3["item_price_level"] = data_3["item_price_level"].astype(str)

In [113]:
data_4 = pd.concat([data_3,pd.get_dummies(data_3[feature_dummy])],axis =1)
data_4 = data_4.drop(feature_dummy,axis = 1)

In [114]:
data_4.shape

(496482, 282)

In [105]:
# feature_std = ['item_price_level','shop_score_description','shop_score_delivery',
#               'shop_score_service','shop_star_level','shop_review_positive_rate',
#               'shop_review_num_level','context_page_id','user_star_level',
#                'item_pv_level','item_collected_level','item_sales_level',
#                'union_property','query_complexity_prope','union_category', 
#                'query_complexity_cate','lenth_cate_pred', 'lenth_prop_pred'
# #               ]
# feature_std = ['shop_score_description','shop_score_delivery','shop_score_service',
#                'shop_review_positive_rate','union_property','lenth_prop',
#                'union_category','lenth_cate_pred','lenth_prop_pred',
#                'context_timestamp',"item_pv_level","user_star_level","context_page_id",
#                "shop_review_num_level","shop_star_level","item_sales_level",
#                "item_collected_level","item_price_level",'prop1', 'prop2', 'prop3', 'prop4', 'prop5', 'prop6',
#                'prop7', 'prop8', 'prop9', 'prop10', 'pred_cate1', 'pred_cate2',
#                'pred_cate3', 'pred_cate4', 'pred_cate5', 'pred_prop1', 'pred_prop2',
#                'pred_prop3', 'pred_prop4', 'pred_prop5', 'pred_prop6', 'pred_prop7',
#                'pred_prop8', 'pred_prop9', 'pred_prop10'
#               ]

In [106]:
# from sklearn.preprocessing import StandardScaler
# std = StandardScaler() 
# std.fit(data_4[feature_std].values)
# data_4[feature_std] = std.transform(list(data_4[feature_std].values))       
# print(data_4.shape)

In [116]:
data_4.head(3)

,item_brand_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,context_timestamp,shop_review_num_level,shop_review_positive_rate,shop_score_service,shop_score_delivery,...,context_page_id_4011,context_page_id_4012,context_page_id_4013,context_page_id_4014,context_page_id_4015,context_page_id_4016,context_page_id_4017,context_page_id_4018,context_page_id_4019,context_page_id_4020
0,1975590437749032870,3,3,4,14,7744,4,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,1975590437749032870,3,3,4,14,14432,4,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,1975590437749032870,3,3,4,14,68652,4,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [118]:
data_5 = data_4.drop("item_brand_id",axis = 1)

In [119]:
#data_4 = data_4.drop(w2v_features,axis = 1)

In [120]:
#std_series = data_4.std(axis = 0)

In [121]:
# import math
# low_var_col = []
# for i in data_4.columns:
#     if std_series[i] < math.sqrt(.8 * (1 - .8)):
#         low_var_col.append(i)

In [122]:
#data_5 = data_4.drop(low_var_col,axis = 1)

In [123]:
#data_4 = data_4.drop("time",axis = 1)

In [124]:
data_5.columns

Index(['item_price_level', 'item_sales_level', 'item_collected_level',
       'item_pv_level', 'context_timestamp', 'shop_review_num_level',
       'shop_review_positive_rate', 'shop_score_service',
       'shop_score_delivery', 'shop_score_description',
       ...
       'context_page_id_4011', 'context_page_id_4012', 'context_page_id_4013',
       'context_page_id_4014', 'context_page_id_4015', 'context_page_id_4016',
       'context_page_id_4017', 'context_page_id_4018', 'context_page_id_4019',
       'context_page_id_4020'],
      dtype='object', length=281)

In [136]:
def test_features(TYPE, data = data_5):
    if TYPE == 'all':
        data_6 = data_5
    elif TYPE == 'NOw2v':
        data_6 = data_5.drop(w2v_features,axis = 1)
    return data_6

In [137]:
data_6 = test_features('NOw2v')

------

In [138]:
train = data_6.loc[:478110,:]
test = data_6.loc[478111:,:]

In [139]:
train.shape

(478111, 256)

In [140]:
test.shape

(18371, 256)

In [141]:
train_y.shape

(478111,)

In [142]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, \
                    ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

In [143]:
MLA = [
#     ensemble.AdaBoostClassifier(),
#     ensemble.BaggingClassifier(),
#     ensemble.ExtraTreesClassifier(),
#     ensemble.GradientBoostingClassifier(),
#     ensemble.RandomForestClassifier(),
#     gaussian_process.GaussianProcessClassifier(),
#     linear_model.LogisticRegressionCV(),
#     linear_model.PassiveAggressiveClassifier(),
#     linear_model.RidgeClassifierCV(),
#     linear_model.SGDClassifier(),
#     linear_model.Perceptron(),
#     svm.SVC(probability=True),
#     svm.NuSVC(probability=True),
    LGBMClassifier(num_leaves=63, max_depth=7, n_estimators=80, n_jobs=20)
    #LGBMClassifier()
    ]

In [144]:
MLA

[LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
         learning_rate=0.1, max_depth=7, min_child_samples=20,
         min_child_weight=0.001, min_split_gain=0.0, n_estimators=80,
         n_jobs=20, num_leaves=63, objective=None, random_state=None,
         reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
         subsample_for_bin=200000, subsample_freq=1)]

In [145]:
train = train.reset_index(drop = True)
test = test.reset_index(drop = True)

In [146]:
cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .7, 
                                        random_state = 42 ) 

MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train log_loss Mean', 
               'MLA Test log_loss Mean' ,'MLA Time']
MLA_compare = pd.DataFrame(columns = MLA_columns)
row_index = 0
for alg in MLA:
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    cv_results = model_selection.cross_validate(alg, train, 
                                                train_y,cv  = cv_split,
                                                scoring='neg_log_loss')

    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'MLA Train log_loss Mean'] = -cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'MLA Test log_loss Mean'] = -cv_results['test_score'].mean()   
    MLA_compare.loc[row_index, 'MLA Test log_loss 3*STD'] = -cv_results['test_score'].std()*3   #let's know the worst that can happen!
    
    alg.fit(train, train_y)
    MLA_predict = alg.predict(test)
    print(MLA_name,"complete...")
    
    row_index+=1

MLA_compare.sort_values(by = ['MLA Test log_loss Mean'], ascending = False, inplace = True)
MLA_compare

i:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


LGBMClassifier complete...


i:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


,MLA Name,MLA Parameters,MLA Train log_loss Mean,MLA Test log_loss Mean,MLA Time,MLA Test log_loss 3*STD
0,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",0.080766,0.0888244,12.013,-0.002423


In [147]:
predict_prob = alg.predict_proba(test)[:,1]

In [152]:
feature_importance = pd.concat([pd.DataFrame(data_6.columns),pd.DataFrame(alg.feature_importances_)],axis =1 )

In [158]:
feature_importance.columns = ["feature", "importance"]

In [163]:
feature_importance.sort_values(by="importance",ascending = False)

,feature,importance
4,context_timestamp,514
18,sales_price_ratio,344
8,shop_score_delivery,279
16,lenth_prop,260
7,shop_score_service,191
10,user_query_day,190
9,shop_score_description,189
11,user_query_day_hour,164
15,union_property,164
6,shop_review_positive_rate,161


In [145]:
submission = pd.read_table("round1_ijcai_18_result_demo_20180301.txt")

In [146]:
submission_final = pd.DataFrame(columns=submission.columns)

In [147]:
submission_final["instance_id"] = test_1["instance_id"]
submission_final["predicted_score"] = predict_prob
submission_final = submission_final.drop("instance_id predicted_score",axis = 1)
submission_final

,instance_id,predicted_score
0,2475218615076601065,0.007260
1,398316874173557226,0.022583
2,6586402638209028583,0.016434
3,1040996105851528465,0.021211
4,6316278569655873454,0.017423
5,868158305045921978,0.022111
6,5713520501786699854,0.018632
7,932945015407923184,0.010509
8,1919197847086752313,0.010476
9,304887065966615346,0.010570


In [148]:
from datetime import date

In [149]:
today = date.today().strftime('%Y%m%d')

In [150]:
submission_final.to_csv("submission_"+today+".csv",index = False,sep = ' ')

In [193]:
submission_final

,instance_id,predicted_score
0,2475218615076601065,0.007404
1,398316874173557226,0.021451
2,6586402638209028583,0.018573
3,1040996105851528465,0.022394
4,6316278569655873454,0.020663
5,868158305045921978,0.020491
6,5713520501786699854,0.013487
7,932945015407923184,0.008327
8,1919197847086752313,0.010393
9,304887065966615346,0.011244
